# Fusion des données des capteurs sur base du timestamp
## Context
Nous avons à notre disposition des données issues des capteurs comme le gyroscope et et accéléromètre. Ces capteurs sont placés sur le dispositif. Ainsi nous avons odes données dans `data/01_raw_input`.

Dans ce dossier, on retrouve 2 types:
- Les données saines, c'est à dire que le dispositif tourne correctement sans perturbation
- Les données avec balourd, c'est à dire que l'on a placé sur le dispositif des perturbateurs qui empêche le dispositif de tourner correctement. Comme par exemple un déséquilibre sur la roue, une distorsion sur un axe, un un ralentissement forcé de l'axe, ou une combinaison de tout ça.

## Objectif
Dans ce code, on va récupérer les données et les combiner sur base de la clé timestamp. On va faire cette opération pour les 2 dossiers séparément. Une fois le traitement fait, ils seront exportés dans le dossier de l'étape suivant, c'est à dire "02_merged".

Les fonctions utilitaires dans ce notebook proviennent de `src/01_fusion_alignement/fusion_alignment.py` afin de simplifier la lecture du code.

## Traitement
En premier temps, nous allons aborder en détails le traitement des données du dossier `PI-donnee-saine`. Les données de `PI-donnee-balourd` seront traités en une seul étape vers la fin.
### 1. Récupération des données de PI-donnee-saine
Tout d'abord, nous allons récupérer les fichiers en spécifiant le dossier cible:

In [1]:
from src.step_01_fusion_alignement.fusion_alignment import get_files, read_and_prepare, process
from pathlib import Path

stable_input = Path("data/01_raw_input/PI-donnee-saine/")
stable_output = Path("data/02_merged/PI-donnee-saine/")
files = get_files(stable_input)
files = get_files(stable_input)

print(files)

df_ax = read_and_prepare(files["ax"], "ax_vibration_ms2")
df_ay = read_and_prepare(files["ay"], "ay_vibration_ms2")
df_az = read_and_prepare(files["az"], "az_vibration_ms2")
df_gx = read_and_prepare(files["gx"], "gx_vibration_gx")
df_gy = read_and_prepare(files["gy"], "gy_vibration_gy")
df_gz = read_and_prepare(files["gz"], "gz_vibration_gz")
df_hall = read_and_prepare(files["hall"], "hall_switch")

{'ax': WindowsPath('data/01_raw_input/PI-donnee-saine/AX_vibration.csv'), 'ay': WindowsPath('data/01_raw_input/PI-donnee-saine/AY_vibration.csv'), 'az': WindowsPath('data/01_raw_input/PI-donnee-saine/AZ_vibration.csv'), 'gx': WindowsPath('data/01_raw_input/PI-donnee-saine/gx_vibration.csv'), 'gy': WindowsPath('data/01_raw_input/PI-donnee-saine/gy_vibration.csv'), 'gz': WindowsPath('data/01_raw_input/PI-donnee-saine/GZ_vibration.csv'), 'hall': WindowsPath('data/01_raw_input/PI-donnee-saine/HALL_switch.csv')}


FileNotFoundError: [Errno 2] No such file or directory: 'data\\01_raw_input\\PI-donnee-saine\\HALL_switch.csv'

### 2. Extraction de toutes les timestamp des fichiers
Étant donnée que le timestamp est différent dans des fichiers, on ne peux pas directement associer les données entre eux. Il faut trouver une astuce. On va commencer par créer un index avec toutes les timestamp.

In [ ]:
import pandas as pd

all_times = pd.Index(sorted(
    pd.concat([df.index.to_series() for df in (df_ax, df_ay, df_az, df_gx, df_gy, df_gz, df_hall)])
    .unique()
))

### 3. Définition d'une tolérence pour le décalage
Ce paramettre permet de résoudre le problème de décalage en ajoutant une tolérance. 

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

delay = os.getenv("STEP1_HORODATAGES_TOLERANCE")# ici 30 millisecondes
tolerance = pd.Timedelta(delay)  

# réindexer chaque df sur all_times en prenant la valeur la plus proche
def reindex_nearest(df, index, tol):
    return df.reindex(index, method="nearest", tolerance=tol)

r_ax = reindex_nearest(df_ax, all_times, tolerance)
r_ay = reindex_nearest(df_ay, all_times, tolerance)
r_az = reindex_nearest(df_az, all_times, tolerance)
r_gx = reindex_nearest(df_gx, all_times, tolerance)
r_gy = reindex_nearest(df_gy, all_times, tolerance)
r_gz = reindex_nearest(df_gz, all_times, tolerance)
r_hall = reindex_nearest(df_hall, all_times, tolerance)

## 4. Formatage et sortie
Après le traitement, les données sont exportées dans le dossier de l'étape suivant pour être utilisé après par le processus suivant.

In [ ]:
# concaténer toutes les colonnes dans un seul DataFrame
df_merged = pd.concat([r_ax, r_ay, r_az, r_gx, r_gy, r_gz, r_hall], axis=1)

# index datetime commun
df_merged.index.name = "date"

# sauvegarde
df_merged.to_csv(stable_output / "PI-donnee-saine_output.csv")

## Recomencement avec "PI-donnee-balourd"
on va exécuter la méthode complète qui exécute toutes les étapes d'avant en une seule fois:

In [ ]:
import src.step_01_fusion_alignement.fusion_alignment as fusion_alignment

process("PI-donnee-balourd")